# Programing exercise 4: Collective Ising spins

Due on Monday, 19.11.2018, 14.15h

### Total: 100 points

## The problem

The collective Ising spin-model with a transverse field is governed by the Hamiltonian
$$
H=-\frac{J}{N}S_z^2 - \Omega S_x
$$
with the collective spin operators $S_\alpha = \sum_i \sigma_i^\alpha/2$, where $\sigma_i^\alpha$ is a Pauli operator acting on spin $i$. This model features a quantum phase transition at $\Omega/J=1$, meaning that the ground state propoerties change abruptly. It also shows a dynamical phase transition at $\Omega/J=0.5$, meaning that the long time properties of the dynamics change drastically at this point. The dynamical case was recently studied experimentally using a chain of trapped ions (https://www.nature.com/articles/nature24654). Note the different convention (x<>z) and the differences in the definition of field and interaction strength chosen there. Also, in the experiment the interactions where not strictly infinite range but powerlaw with a small exponent.

We want to use this model to see how phase transitions can be observed numerically by extrapolating to infinite $N$, and to see how symmetries can be exploited to greatly reduce the size of the relevant Hilbert space of a spin system.

In [21]:
# load standard libraries

import numpy as np   # standard numerics library
import numpy.linalg as LA
import scipy.linalg as sciLA

import matplotlib.pyplot as plt   # for making plots

import time as time

import scipy.sparse as sp
import scipy.sparse.linalg as sLA

%matplotlib inline

### Exercise 1 (25 points)

We first want to study the low lying eigenstates and observe the quantum pahse transition. Thus, it makes sense to work with sparse matrices.

Generate the Hamiltonian as a sparse matrix. You can proceed similarly to the coupled oscillator problem: Build the matrix representation of the collective spin operator $S_+$ and $S_z$, and then use these to construct any other operators you need, e.g. $S_x=(S_+ + S_-)/2$. For matrix-matrix and matrix-vector multiplication it is intuitive to use the @ operator in python 3, which is a short hand for the dot() function.

Test your implementation for N=20 with small and large values of $\Omega$ (set J=1 from now on, meaning that times are in units of 1/J and when we speak of $\Omega$ we really mean the dimensionless parameter $\Omega/J$). What do you expect for $S_z$ and $S_x$ in either limit? 

Collective Spin operator: $$S_a=\sum_{i=1}^N S_a^{(i)}=\sum_{i=1}^N \sigma_a^{(i)}/2=\frac{1}{2^N}\ \sigma_a^{(1)} \otimes \dots \otimes \sigma_a^{(N)}$$ where $$\sigma_a^{(i)}=\mathbf 1 \otimes \dots \otimes \sigma_a \otimes \dots \otimes \mathbf 1$$ $\sigma_a$ at index $i$.

In [91]:
# spin matrices as sparse matrices
s = [sparse.csr_matrix(np.array([[0, 1]  ,[ 1, 0]])/2),    #sx
     sparse.csr_matrix(np.array([[0, -1j],[1j, 0]])/2),    #sy
     sparse.csr_matrix(np.array([[1, 0]  ,[0, -1]])/2)]    #sz
#function to compute N kronecker products of identities with 2x2 matrices at certain indices
def NKron(N,matrices=[],indices=[]):
    result=matrices[0] if 0 in indices else sp.identity(2)
    for i in range(1,N):
        result=sp.kron(result,matrices[indices.index(i)] if i in indices else sp.identity(2),format='csr')
    return result
#return the collective spin operator. a is in [0,1,2] for (x,y,z). N is particle number
def collectiveSpin(N,a):
    return NKron(N,[s[a]]*N,[*range(N)])
#return the squared collective Spin
def collectiveSpin2(N,a):
    result=sp.csr_matrix((2**N,2**N))
    for i in range(N):
        for j in range(i):
            result+=NKron(N,[s[a]]*2,[i,j])
    return result*2

In [105]:
#Hamiltonian as sparse matrix:
def H(N,Om,J=1):
    #-J/N*S_z^2-Ω*S_x
    return -J/N*collectiveSpin2(N,2)-Om*collectiveSpin(N,0)

### Exercise 2 (20 points)

Loop over $\Omega$ (from 0 to 2 in small steps) and record the lowest 3 eigenenergies and, for the ground state, the probabilities P(M) to be in state $|N/2,M\rangle$. Plot the 3 lowest energy levels and P(M;$\Omega$) (density plot!). At small $\Omega$ you should observe kind of a "spontaneous symmetry breaking": The gap between ground and first excited state becomes very small. The states $|00...0\rangle = |N/2,N/2\rangle$ and $|11...1\rangle = |N/2,-N/2\rangle$ are degenerate ground states at a $\Omega=0$. The Hamiltonian has a $Z_2$ symmetry. It is invariant under flipping of all spins. Thus the eigenstates should also respect this symmetry and be either symmetric or anti-symmetric with respect to spin flips. However, when the gap between the symmetric and anti-symmetric state becomes too small, the numerical diagonalization algorithm just randomly choses a superposition of the two states, which can as well be $|N/2,-N/2\rangle$ or $|N/2,N/2\rangle$, thus breaking the symmetry.

The spin flip symmetry means that, if written in the symmetrized basis, the Hamiltonian will consist of two disconnected blocks, representing the symmetric ($\propto |N/2,M\rangle + |N/2,-M\rangle$) and anti-symmetric states ($\propto |N/2,M\rangle - |N/2,-M\rangle$). This means that the relevant gap we have to look at is the gap between lowest two states of one block. If we don't want to do the symmetrization explicitly, this can be done by just looking at the energy difference between the ground and the second excited state, which should be both symmetric.

[Optional]: One could also fix this by considering the symmetric and anti-symmetric blocks separately. For this you need to implement the Hamiltonian again in the symmetrized basis (or use a basis transformation into the symmetrized basis). This will give a speedup of at least a factor of 2.

### Exercise 3 (20 points)

Loop over omega and go to large N to see how the gap ($E_2-E_0$) closes at the critical point ("finite size scaling analysis"). Extract the position and size of the smallest gap as a function of N. Loop N from 40 to 400 in steps of 20. The easiest way to get the minimal gap is to calculate it on an omega-gid around 1, where the transition is expected to be. I increased $\Omega$ from 0.8 to 1 in 100 steps. You could also make the search for the minimal gap more efficient by implementing an iterative search procedure.

Plot the resulting distance of the optimal $\Omega$ and size of the minimal gap double logarithmically. You should get a power law behavior. Extract the exponent of this power law for both quantities.

### Exercise 4 (20 points)

Now calculate the time evolution under the above Hamiltonian, starting in the state $|00...0\rangle = |N/2,N/2\rangle$. For this, you could use scipy's expm function which allows you to simply exponentiate a matrix so you can directly calculate $exp(-iHt)$ and apply it to the initial state (e.g. using the @ operator). Try this out. You will see that it becomes really slow as you go to larger atom numbers. The more efficient way is to use exact diagonalization. This time the use of sparse matrices is not crucial since we need all the eigenstates anyway. Also, the spin flip symmetry is now not so useful since our initial state already breaks this symmetry. 

To calculate $|\psi(t)\rangle$, project the initial state onto the eigenstates. Remember that the eigenstates are in the columns of the eigenvector matrix returned by eig (or eigh). Thus the transformation into the energy eigenbasis can be achieved by applying the conjugate transpose of this matrix to the initial state. Evolve the state in the eigenbasis and transform back.

Calculate the expectation values of $S_z$ and $S_z^2$ as a function of time for some value of $\Omega$ below and above 0.5. You should see different behavior in the two regimes. Check whether your reults argee qualitatively with the observations in https://www.nature.com/articles/nature24654 in particular Fig. 2 (the paper is also on the arxiv at https://arxiv.org/abs/1708.01044). Note that their definition of the parameter $\Omega/J$ differs by a factor 1/2 from ours such that the transition point is at 1 for them.

### Exercise 5 (15 points)

Study the long-time limit systematically as a function of $\Omega$. Loop $\Omega$ from 0 to 1 in at least 50 steps. Use at least N=100. Calculate the values of $S_z$ and $S_z^2$ at some long time. Also calculate the infinite time average 
$$
\langle O \rangle_\infty = \sum_k |\langle \psi_0|\phi_k\rangle|^2 \langle \phi_k|O|\phi_k\rangle
$$
where $|\phi_k\rangle$ are the eigenstates and $O$ is the observables of interest (here $S_z$ or $S_z^2$).
Compare your results to Fig. 6 in https://arxiv.org/abs/1708.01044. This shows the dynmaical phase transition.